In [1]:
from torchdrug.tasks import NodePropertyPrediction
from torchdrug import transforms
from torchdrug import data, core, layers, tasks, metrics, utils, models
from torchdrug.layers import functional
from torchdrug.core import Registry as R
from torchdrug.models import GearNet

import torch
from torch.utils import data as torch_data
from torch.nn import functional as F

from lib.datasets import ATPBind

truncuate_transform = transforms.TruncateProtein(max_length=350, random=False)
protein_view_transform = transforms.ProteinView(view='residue')
#transform = transforms.Compose([truncuate_transform, protein_view_transform])
transform = transforms.Compose([protein_view_transform])

dataset = ATPBind(atom_feature=None, bond_feature=None,
                  residue_feature="default", transform=transform)

train_set, valid_set, test_set = dataset.split()
print("train samples: %d, valid samples: %d, test samples: %d" %
      (len(train_set), len(valid_set), len(test_set)))


Split num:  [346, 42, 41]
train samples: 346, valid samples: 42, test samples: 41


In [21]:
model = models.GearNet(input_dim=21, hidden_dims=[512, 512, 512], 
                              num_relation=7, edge_input_dim=59, num_angle_bin=8,
                              batch_norm=True, concat_hidden=True, short_cut=True, readout="sum")

In [6]:
protein = train_set[0]['graph']
protein

Protein(num_atom=0, num_bond=0, num_residue=566)

In [9]:
model(protein, protein.node_feature.float())

RuntimeError: addmm: Argument #3 (dense): Expected dim 0 size 0, got 566

In [10]:
# read protein from pdb file



Protein(num_atom=4769, num_bond=9746, num_residue=597)


/data/project/aigenintern/aigenintern2/miniconda3/envs/jc/lib/python3.9/site-packages/torchdrug/data/protein.py:213: UserWarning: Unknown residue `AMP`. Treat as glycine
  warnings.warn("Unknown residue `%s`. Treat as glycine" % type)
/data/project/aigenintern/aigenintern2/miniconda3/envs/jc/lib/python3.9/site-packages/torchdrug/data/feature.py:42: UserWarning: Unknown value `AMP`
  warnings.warn("Unknown value `%s`" % x)
/data/project/aigenintern/aigenintern2/miniconda3/envs/jc/lib/python3.9/site-packages/torchdrug/data/protein.py:213: UserWarning: Unknown residue `ATP`. Treat as glycine
  warnings.warn("Unknown residue `%s`. Treat as glycine" % type)
/data/project/aigenintern/aigenintern2/miniconda3/envs/jc/lib/python3.9/site-packages/torchdrug/data/feature.py:42: UserWarning: Unknown value `ATP`
  warnings.warn("Unknown value `%s`" % x)
/data/project/aigenintern/aigenintern2/miniconda3/envs/jc/lib/python3.9/site-packages/torchdrug/data/protein.py:213: UserWarning: Unknown residue `M

In [11]:
model(protein, protein.node_feature.float())


RuntimeError: shape '[4769, 84]' is invalid for input of size 57228

In [24]:
from torchdrug import layers
from torchdrug.layers import geometry

graph_construction_model = layers.GraphConstruction(
                                edge_layers=[geometry.SpatialEdge(radius=10.0, min_distance=5),
                                                                 geometry.KNNEdge(k=10, min_distance=5),
                                                                 geometry.SequentialEdge(max_distance=2)],
                                                    edge_feature="gearnet")


file_path = 'data/script/pdb/3EPSA.pdb'
protein = data.Protein.from_pdb(
    file_path, atom_feature="position", bond_feature="length", residue_feature="symbol")
print(protein)
print(protein.num_relation)

protein = data.Protein.pack([protein])
protein = graph_construction_model(protein)
print(protein)
print(protein.num_relation)

/data/project/aigenintern/aigenintern2/miniconda3/envs/jc/lib/python3.9/site-packages/torchdrug/data/protein.py:213: UserWarning: Unknown residue `AMP`. Treat as glycine
  warnings.warn("Unknown residue `%s`. Treat as glycine" % type)
/data/project/aigenintern/aigenintern2/miniconda3/envs/jc/lib/python3.9/site-packages/torchdrug/data/feature.py:42: UserWarning: Unknown value `AMP`
  warnings.warn("Unknown value `%s`" % x)
/data/project/aigenintern/aigenintern2/miniconda3/envs/jc/lib/python3.9/site-packages/torchdrug/data/protein.py:213: UserWarning: Unknown residue `ATP`. Treat as glycine
  warnings.warn("Unknown residue `%s`. Treat as glycine" % type)
/data/project/aigenintern/aigenintern2/miniconda3/envs/jc/lib/python3.9/site-packages/torchdrug/data/feature.py:42: UserWarning: Unknown value `ATP`
  warnings.warn("Unknown value `%s`" % x)
/data/project/aigenintern/aigenintern2/miniconda3/envs/jc/lib/python3.9/site-packages/torchdrug/data/protein.py:213: UserWarning: Unknown residue `M

Protein(num_atom=4769, num_bond=9746, num_residue=597)
tensor(4)
PackedProtein(batch_size=1, num_atoms=[4769], num_bonds=[267424], num_residues=[597])
tensor(7)


In [23]:
model(protein, protein.node_feature.float())


RuntimeError: shape '[566, 147]' is invalid for input of size 11886